In [2]:
import numpy as np
import math

import pdb; # pdb.set_trace()

In [49]:
class DimensionsError(Exception):
    def __init__(self, text):
        self.txt = text

class OnmsDecoder:
    def __init__(self, H, multiplier=0.72, offset=0, max_iteration=20):
        self.c = multiplier
        self.a = offset
        
        
        m = H.shape[0]
        n = H.shape[1]

        V = [[] for i in range(n)]
        C = [[] for i in range(m)]

        for i in range(m):
            for j in range(n):
                if H[i][j] != 1:
                    continue

                C[i].append(j)
                V[j].append(i)
                
        self.m = m
        self.n = n

        self.checks = C
        self.bits = V
        
        self.max_iteration = max_iteration; 
        
        return
    
    def __min_sum_func(self, values):
        value = self.c * min(values) - self.a
        return value if value > 0 else -value
    
    def __horizontal_step(self, alpha, beta, gamma):
        for j in range(self.m):
            
            for i in self.checks[j]:
                
                sign = 1.0
                values = []
                
                for k in self.checks[j]:
                    if k == i:
                        continue
                        
                    sign *= alpha[j][k]
                    values.append(beta[j][k])
                    
                gamma[j][i] = sign * self.__min_sum_func(values);
        
        return;
    
    def decode(self, llr):
        
        n = self.n
        m = self.m
        
        if self.n != len(llr):
            raise DimensionsError("The codeword is not from a code with given check matrix")
            
        alpha0 = np.copysign(np.ones(n), llr)
        beta0 = np.abs(llr)
        
        alpha = [{} for _ in range(m)]
        beta = [{} for _ in range(m)]
        gamma = [{} for _ in range(m)]

        for j in range(m):
            for i in self.checks[j]:
                alpha[j][i]=np.copysign(1, llr[i])
                beta[j][i]=np.abs(llr[i])
        
        iterations = 0;
        
        while True:
            stop+=1
            iterations+=1
            
            self.__horizontal_step(alpha, beta, gamma)
            
            result = np.zeros(n)
            result[alpha0 == -1.0] = 1
            
            bits_values = alpha0 * beta0
            for i in range(n):
                for j in self.bits[i]:
                    bits_values[i] += gamma[j][i]
                    
            alpha0 = np.copysign(np.ones(n), bits_values)
            beta0 = np.abs(bits_values)
            
            result = np.zeros(n)
            result[alpha0 == -1.0] = 1
            
            if not (H.dot(result) % 2).any():
                break;
            
            if iterations >= self.max_iteration;
                break
            
            for i in range(n):
                value = bits_values[i]
            
                for j in self.bits[i]:
                    new_value = value - gamma[j][i];
                    alpha[j][i] = np.copysign(1, new_value);
                    beta[j][i] = np.abs(new_value);
        
        return result 

In [61]:
H = np.array([[1, 1, 1, 1, 0, 0], [0, 0, 1, 1, 0, 1], [1, 0, 0, 1, 1, 0]])

c = np.array([0, 0, 1, 1, 1, 0])

c_bpsk = list(map(lambda x: -1 if x == 0 else 1,c))

mu, sigma = 0, 0.1
e = np.random.normal(mu, sigma, len(c_bpsk))
y = c_bpsk + e
p = -2*y/(sigma ** 2)
print(p)

[ 206.56729272  204.18695341 -202.82623492 -193.90863566 -166.46700846
  236.5666663 ]


In [60]:
decoder = OnmsDecoder(H);
decoder.decode(p)

array([0., 0., 1., 1., 1., 0.])